### CREATE DATAFRAME

In [ ]:
from cloudvolume import CloudVolume
from meshparty import skeletonize, trimesh_io
from caveclient import CAVEclient
import trimesh
import numpy as np
import datetime
import networkx as nx
from scipy.sparse import identity
from scipy.spatial import distance_matrix
import scipy 
from tqdm import tqdm
# import aws
import pandas as pd
import csv
import pyembree
import matplotlib.pyplot as plt
import scipy.spatial as spatial
import itertools

In [34]:

orphans = pd.read_csv("/Users/sheeltanna/Desktop/AGT_REPO/campfire/GT_30_Orphans_Spring_2023 - Sheet1.csv")

In [35]:
def my_array(x):
    res = list(map(str.strip, x.split('; ')))
    return res

In [36]:
orphans['endpoints'] = orphans['endpoints'].map(lambda x: list(map(str.strip, x.split('; '))))

In [37]:
## convert from string list to 2-d array
def convert_to_array(row):
    count = 0
    result = []
    for endpoint in row["endpoints"]:
        # endpoint = tuple(map(int, endpoint.split(', ')))
        endpoint = eval(endpoint)
        # print()
        # print(endpoint)
        # print(type(endpoint))
        if(count == 0):
            result = np.array(endpoint)
            count = count + 1
        else:
            result = np.vstack((result, np.array(endpoint)))
            count = count + 1
            #result = np.concatenate(result, list(tuple))
    #check if there was only 1 point, convert to 2-d array:
    # if(type(result) == list):
        
    if(count == 1 and result.size != 0):
        result = result.reshape(1,3)
    return result 

In [38]:
orphans["real_endpoints"] = orphans.apply(convert_to_array, axis = 1)

In [40]:
orphans["real_endpoints"]

0     [[402547, 228831, 23991], [402958, 229220, 235...
1     [[401602, 224623, 23991], [405273, 226312, 236...
2     [[403332, 227545, 24398], [403566, 227614, 244...
3     [[77262, 113088, 20442], [79178, 109113, 20400...
4     [[100423, 143992, 21653], [100583, 144194, 217...
5      [[71894, 146181, 20442], [71498, 146290, 20403]]
6       [[77575, 107807, 21133], [77000, 99565, 21215]]
7     [[79785, 135709, 21133], [84176, 127514, 21582...
8       [[81198, 106660, 21133], [88834, 99501, 20479]]
9      [[75409, 116393, 21135], [74841, 116076, 21217]]
10    [[351150, 143681, 15176], [348147, 144970, 148...
11    [[344600, 142349, 16939], [344592, 142335, 169...
12    [[366719, 138890, 16417], [367885, 138975, 165...
13    [[353411, 147608, 15177], [352173, 148914, 148...
14    [[346497, 82778, 27090], [339051, 74578, 26769...
15     [[351839, 91058, 27284], [349567, 97065, 26976]]
16    [[357525, 81584, 25828], [358640, 81180, 25734...
17     [[347271, 87374, 25828], [347862, 89018, 